In [1]:
#Author: Anshul Kapoor
#Code Purpose: Make Single JSON for all EGamma SFs

#All the files for SFs
files=[
    "EGdata/egammaEffi.txt_Ele_Veto_postVFP_EGM2D.root",
    "EGdata/egammaEffi.txt_Ele_Loose_postVFP_EGM2D.root",
    "EGdata/egammaEffi.txt_Ele_Medium_postVFP_EGM2D.root",
    "EGdata/egammaEffi.txt_Ele_Tight_postVFP_EGM2D.root",
]

#Names that you want for SFs for files above (same order please)
names=["Veto","Loose","Medium","Tight"]
#Names that you want for errors for files above (same order please)
nameserrs=["Veto_err","Loose_err","Medium_err","Tight_err"]
nameJSON="egcorrs_withErrors.json" # Name of final JSON

sfhist="EGamma_SF2D"

In [2]:
from ROOT import TFile, TH1F, TCanvas, TString

#Function to extract SFs from EGamma standard root files
def getSFs(fn="filename"):
    tf = TFile(fn)
    fo = tf.Get(sfhist)
    Xbins=fo.GetNbinsX()
    Ybins=fo.GetNbinsY()
    X=[None]*(Xbins+1)
    Y=[None]*(Ybins+1)
    values=[]
    errors=[]
    for i in range(1,Xbins + 1):
        X[i-1]=(fo.GetXaxis().GetBinLowEdge(i))
    X[Xbins]=fo.GetXaxis().GetBinUpEdge(Xbins)
    for j in range(1,Ybins + 1):
        Y[j-1]=(fo.GetYaxis().GetBinLowEdge(j))
    Y[Ybins]=fo.GetYaxis().GetBinUpEdge(Ybins)
    for i in range(1,Xbins + 1):
        for j in range(1,Ybins + 1):
            values.append(fo.GetBinContent(i,j))
            errors.append(fo.GetBinError(i,j))
    return X,Y,values,errors


#Use correctionlib to prepare jsons for all wps
from correctionlib.schemav1 import Correction
corrs=[]
for name,name_err,file in zip(names,nameserrs,files):
    X,Y,values,errors=getSFs(file) #Using getSFs function defined above
    
    corr = Correction.parse_obj(
    {
        "version": 1,
        "name": name,
        "inputs": [
            {"type": "real", "name": "eta", "description": "possibly supercluster eta?"},
            {"name": "pt", "type": "real"},
        ],
        "output": {"name": "weight", "type": "real"},
        "data": {
            "nodetype": "multibinning",
            "edges": [
                X,
                Y,
            ],
            "content": values,
        },
    })
    corrs.append(corr)
    
    correrr = Correction.parse_obj(
    {
        "version": 1,
        "name": name_err,
        "inputs": [
            {"type": "real", "name": "eta", "description": "possibly supercluster eta?"},
            {"name": "pt", "type": "real"},
        ],
        "output": {"name": "weight", "type": "real"},
        "data": {
            "nodetype": "multibinning",
            "edges": [
                X,
                Y,
            ],
            "content": errors,
        },
    })
    corrs.append(correrr)

#Save JSON
from correctionlib.schemav1 import CorrectionSet
import gzip

cset = CorrectionSet.parse_obj({
    "schema_version": 1,
    "corrections": corrs,
})

with open(nameJSON, "w") as fout:
    fout.write(cset.json(exclude_unset=True, indent=4))

Welcome to JupyROOT 6.22/06


In [3]:
#Evaluator Example

import libcorrection

cset = libcorrection.CorrectionSet(nameJSON)

valsf= cset["Veto"].evaluate(1.1, 34.0)
valsferr= cset["Veto_err"].evaluate(1.1, 34.0)

print("SF is:"+str(valsf))
print("error is:"+str(valsferr))

SF is:0.9864253401756288
error is:0.013436816646698298
